In [1]:
import akshare as ak
import pandas as pd
import datetime

In [2]:
def industry_info(days=29):
    """
    Extracts the main net flow and change percentage of each industry over the last `days` days.
    :param days: Number of days to consider for net flow calculation.
    :return: DataFrame with industry names, their main net flow, and change percentage.
    """
    # Get the list of industry names
    stock_board_industry_name_em_arr = ak.stock_board_industry_name_em()["板块名称"]
    # Get the first and last date for the specified number of days
    first_date = ak.stock_sector_fund_flow_hist(symbol="证券").iloc[-days]["日期"]
    first_date_str = first_date.strftime("%Y%m%d")
    last_date = ak.stock_sector_fund_flow_hist(symbol="证券").iloc[-1]["日期"]
    last_date_str = last_date.strftime("%Y%m%d")
    # Initialize a pandas Dataframe to hold industry names, industry main net flow, and industry index change percentage
    df = pd.DataFrame(columns=["行业名称", "主力净流入-总净额", "行业涨跌幅"])
    # Loop through each industry
    for industry_name in stock_board_industry_name_em_arr:
        # Extract industry name
        stock_sector_fund_flow_hist_df = ak.stock_sector_fund_flow_hist(symbol=industry_name).iloc[-days:]
        # Calculate main net flow
        industry_main_net_flow = stock_sector_fund_flow_hist_df["主力净流入-净额"].sum()
        # Calcuate change percentage
        industry_hist_index = ak.stock_board_industry_hist_em(symbol=industry_name, start_date=first_date_str, end_date=first_date_str, period="日k", adjust="").iloc[0]["收盘"]
        industry_today_index = ak.stock_board_industry_hist_em(symbol=industry_name, start_date=last_date_str, end_date=last_date_str, period="日k", adjust="").iloc[0]["收盘"]
        industry_index_change_percentage = (industry_today_index - industry_hist_index) / industry_hist_index * 100
        industry_index_change_percentage = round(industry_index_change_percentage, 2)
        print(f"{industry_name}: {industry_main_net_flow}, {industry_index_change_percentage}%")
        # Append the results to the DataFrame
        df.loc[len(df)] = [industry_name, industry_main_net_flow, industry_index_change_percentage]
    return df

# # 主力净流入-总净额 > 50 亿, 并且行业涨跌幅 < 8%
# df_filtered = df[(df["主力净流入-总净额"] > 5e9) & (df["行业涨跌幅"] < 8)]
# df_filtered

In [3]:
industry_info_df = industry_info(days=29)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

船舶制造: -6317680448.0, 17.97%
小金属: -6950252432.0, 18.94%
证券: 5136258208.0, 13.09%
多元金融: -271065360.0, 25.99%
软件开发: -29420723504.0, 11.39%
贸易行业: -2648144433.0, 8.69%
互联网服务: -23248636080.0, 10.44%
有色金属: -8363104864.0, 11.6%
医疗服务: -5325389968.0, 8.26%
生物制品: -6716129936.0, 5.76%
汽车服务: -526485768.0, 4.92%
非金属材料: -824826147.0, 13.05%
贵金属: -4094884096.0, 5.33%
电机: -3298242879.0, 5.08%
化学制药: -21141113888.0, 7.53%
半导体: -22800459584.0, 6.65%
能源金属: -555705113.0, 14.03%
钢铁行业: 514943376.0, 13.08%
计算机设备: -12874643680.0, 8.8%
专业服务: -1914674013.0, 10.5%
石油行业: -2427497440.0, 7.76%
房地产开发: -2327435424.0, 9.88%
通用设备: -22690558416.0, 5.52%
专用设备: -26213457264.0, 7.63%
航天航空: -12842400384.0, 9.39%
综合行业: -1612794895.0, 10.26%
工程机械: -3586548080.0, 4.22%
汽车零部件: -30338997296.0, 3.58%
农牧饲渔: -6738783760.0, 3.11%
航空机场: -1062698812.0, -0.4%
通信设备: -20115830784.0, 11.51%
酿酒行业: -12738338288.0, -0.31%
仪器仪表: -3323753166.0, 6.98%
医疗器械: -4931589808.0, 5.07%
中药: -6477581744.0, 2.2%
化肥行业: -3269532793.0, 8.45%
工程建设: -3603747888.

In [4]:
filtered_industries = industry_info_df[(industry_info_df["主力净流入-总净额"] > 5e9) & (industry_info_df["行业涨跌幅"] < 8)]
filtered_industries = filtered_industries["行业名称"]

In [5]:
industry_info_df.to_csv("industry_info.csv", index=False)

In [6]:
def stock_analysis(industry_name, days=29):
    """
    Analyzes stocks in a given industry by extracting their main net inflow and price change percentage over the last `days` days.
    :param industry: The industry name to analyze.
    :param days: Number of days to consider for net flow calculation.
    :return: None, prints the stock name, main net inflow, and price change
    """
    # Extract all stocks from the industry board
    stock_board_industry_cons_em_df = ak.stock_board_industry_cons_em(symbol=industry_name)
    # Extract stock names and codes from the industry board
    stock_info = stock_board_industry_cons_em_df[["代码", "名称"]]

    # Initialize a pandas Dataframe to hold industry names, industry main net flow, and industry index change percentage
    df = pd.DataFrame(columns=["行业", "代码", "名称", "主力净流入-总净额", "涨跌幅"])
    for row in stock_info.itertuples():
        stock_code = row.代码
        stock_name = row.名称
        # Determine the market based on the stock code
        if stock_code.startswith("6"):
            market = "sh"
        elif stock_code.startswith("0") or stock_code.startswith("3"):
            market = "sz"
        else:
            market = "bj"
        
        # Extract the historical data of the stock
        stock_individual_fund_flow_df = ak.stock_individual_fund_flow(stock=stock_code, market=market).iloc[-days:]
        # Get the main net inflow data for the last 29 days
        stock_main_net_flow = stock_individual_fund_flow_df["主力净流入-净额"].sum()
        # Calculate change percentage
        stock_hist_price = stock_individual_fund_flow_df.iloc[-days]["收盘价"]
        stock_today_price = stock_individual_fund_flow_df.iloc[-1]["收盘价"]
        stock_price_change_percentage = (stock_today_price - stock_hist_price) / stock_hist_price * 100
        stock_price_change_percentage = round(stock_price_change_percentage, 2)
        
        print(f"{stock_name}: {stock_main_net_flow}, {stock_price_change_percentage}%")
        # Append the results to the DataFrame
        df.loc[len(df)] = [industry_name, stock_code, stock_name, stock_main_net_flow, stock_price_change_percentage]

    return df

In [ ]:
df = pd.DataFrame(columns=["行业", "代码", "名称", "主力净流入-总净额", "涨跌幅"])
for industry_name in filtered_industries:
    stock_analysis_df = stock_analysis(industry_name)
    df = pd.concat([df, stock_analysis_df], ignore_index=True)

0it [00:00, ?it/s]

新华保险: 311678460.0, 20.45%
中国太保: 1722278819.0, 11.48%
中国平安: 3027649888.0, 10.45%
中国人寿: 410091318.0, 5.11%
中国人保: 482473178.0, 1.92%
*ST天茂: -297147438.0, -42.38%


/var/folders/bn/xnw38wz5697c2m0cykr9528m0000gn/T/ipykernel_34953/4101310503.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, stock_analysis_df], ignore_index=True)


In [8]:
stock_analysis_df

,行业,代码,名称,主力净流入-总净额,涨跌幅
0,保险,601336,新华保险,3.116785e+08,20.45
1,保险,601601,中国太保,1.722279e+09,11.48
2,保险,601318,中国平安,3.027650e+09,10.45
3,保险,601628,中国人寿,4.100913e+08,5.11
4,保险,601319,中国人保,4.824732e+08,1.92
5,保险,000627,*ST天茂,-2.971474e+08,-42.38


In [9]:
df_filtered = df[(df["主力净流入-总净额"] > 1e8) & (df["涨跌幅"] < 10)]
df_filtered

,行业,代码,名称,主力净流入-总净额,涨跌幅
3,保险,601628,中国人寿,410091318.0,5.11
4,保险,601319,中国人保,482473178.0,1.92


In [10]:
df.to_csv("stock_analysis.csv", index=False)